In [ ]:
import os

os.environ["HF_HOME"] = "/root/autodl-tmp/HF_download"
os.environ["MODELSCOPE_CACHE"] = "/root/autodl-tmp/MODELSCOPE_download"
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [ ]:
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    # torch_dtype=torch.bfloat16
).to("cuda")

In [ ]:
from datasets import load_dataset

datasets = load_dataset("Moemu/Muice-Dataset")

In [ ]:
import re


def process2messages_function(examples):
    messages = []
    for system, conversation in zip(examples["system"], examples["conversation"]):
        message = []
        message.append({"role": "system", "content": system})
        for message_pair in conversation:
            message.append({"role": "user", "content": message_pair["human"]})
            message.append({"role": "assistant", "content": message_pair["assistant"]})

        message = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=False
        )
        message = re.sub(r"<think>[\s\S]*?</think>", "", message)

        messages.append(message)
    return {"messages": messages}


In [ ]:
import re


def find_assistant_content_including_end(text):
    pattern = r"<\|im_start\|>assistant\n(.*?<\|im_end\|>)"
    spans = []
    for match in re.finditer(pattern, text, flags=re.DOTALL):
        start = match.start(1)  # 第一个括号组的开始
        end = match.end(1) - 1  # 左闭右闭
        spans.append((start, end))
    return spans


def process_messages2ids_function(examples):
    inputs = tokenizer(
        examples["messages"], truncation=True, max_length=4096, return_offsets_mapping=True
    )
    offset_mapping = inputs.pop("offset_mapping")
    labels = []

    for batch_idx in range(len(inputs["input_ids"])):
        offsets = offset_mapping[batch_idx]
        input_ids = inputs["input_ids"][batch_idx]
        label = [-100] * len(input_ids)

        assistant_contents_idxes = find_assistant_content_including_end(examples["messages"][batch_idx])
        assistant_contents_i = 0
        for idx, offset in enumerate(offsets):
            if assistant_contents_idxes[assistant_contents_i][0] <= offset[0] and offset[1] <= \
                    assistant_contents_idxes[assistant_contents_i][1] + 1:
                label[idx] = input_ids[idx]
            if offset[1] >= assistant_contents_idxes[assistant_contents_i][1] + 1:
                assistant_contents_i += 1
                if assistant_contents_i == len(assistant_contents_idxes):
                    break
        labels.append(label)

    inputs["labels"] = labels

    return inputs

In [ ]:
tokenized_datasets = (datasets.map(process2messages_function, batched=True, remove_columns=datasets["train"].column_names)
                      .map(process_messages2ids_function, batched=True, remove_columns=["messages"]))

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=2,
    # bf16=True,
    report_to=["tensorboard"],
    # gradient_checkpointing=True,
    # pip install bitsandbytes
    # optim="adamw_bnb_8bit",
)

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, do_sample=True)

In [ ]:
pipe.generation_config

In [ ]:
# pipe.generation_config.eos_token_id = 151645

In [ ]:
messages = [
    {"role": "system", "content": "你是一个名为沐雪的可爱AI女孩子"},
    {"role": "user", "content": "你好"}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

pipe(prompt)